In [1]:
from csv_edit import *
from df_edit import *

In [ ]:
!pip install googletrans==4.0.0-rc1

In [1]:
from googletrans import Translator

translator = Translator()

In [13]:
print(translator.translate('호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으로 명칭을 통일하였다.', src='ko', dest='en'))

Translated(src=ko, dest=en, text=Honam-based, the right futures, alternative and Democratic Peace, were worthy of the end of the twist, and unified the name (tentative)., pronunciation=None, extra_data="{'confiden...")


In [2]:
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

<ipython-input-2-3c150f134808>:6: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


#### https://me2.kr/0j4cp 보고 따라하고 Google이 나오면 됩니다.

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome import service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

options = Options()
options.add_argument("--no-sandbox") #bypass OS security model
options.add_argument("--headless")
options.add_argument("--disable-dev-shm-usage") #overcome limited resource problems

s=Service('/opt/ml/chromedriver')
driver = webdriver.Chrome(options=options, service=s)
driver.get('http://google.com/')
myPageTitle = driver.title
print(myPageTitle)
driver.quit()

Google


In [39]:
train_df = pd.read_csv('/opt/ml/dataset/train/train.csv')
train_df_2 = better_df(train_df)

In [40]:
(len(train_df_2))

32470

In [5]:
for i in range(len(train_df_2)):
    if train_df_2['sentence'].iloc[i][train_df_2['obj_start'].iloc[i]:train_df_2['obj_end'].iloc[i]+1] != train_df_2['obj_word'].iloc[i]:
        print(train_df_2['sentence'].iloc[i])
        print(train_df_2['subj_word'].iloc[i])

In [41]:
sen_dict = train_df_2.to_dict()
sub_ent_list = []
obj_ent_list = []

for i in range(train_df_2.shape[0]):
    sub_ent_list.append((train_df_2['subj_start'].iloc[i],train_df_2['subj_end'].iloc[i]))
    obj_ent_list.append((train_df_2['obj_start'].iloc[i],train_df_2['obj_end'].iloc[i]))

In [45]:
len(sen_dict['sentence'])

32470

In [46]:
sub_code = "VKV" # entity가 사라지는 것을 방지
obj_code = "VJV"
encode_dict={}
for idx in range(len(sen_dict['sentence'])):
    sub_start = sub_ent_list[idx][0]
    sub_end = sub_ent_list[idx][1]
    obj_start = obj_ent_list[idx][0]
    obj_end = obj_ent_list[idx][1]
    if sub_start<obj_start:
        encode_dict[idx]=(sen_dict['sentence'][idx][:sub_start]+sub_code+sen_dict['sentence'][idx][sub_end+1:obj_start]+obj_code+sen_dict['sentence'][idx][obj_end+1:])
    else :
        encode_dict[idx]=(sen_dict['sentence'][idx][:obj_start]+obj_code+sen_dict['sentence'][idx][obj_end+1:sub_start]+sub_code+sen_dict['sentence'][idx][sub_end+1:])

In [49]:
# 인코딩 확인
encode_dict[0]

'〈Something〉는 VJV이 쓰고 VKV가 1969년 앨범 《Abbey Road》에 담은 노래다.'

In [51]:
for key,item in encode_dict.items():
    if len(item)>200:
        quotioent = len(item) // 200
        new_line_idx_list=[]
        for i in range(quotioent):
            new_line_idx = 200*(i+1)
            while new_line_idx < len(item) and item[new_line_idx] != ' ':
                new_line_idx += 1
            new_line_idx_list.append(new_line_idx)
        for n_idx in new_line_idx_list:
            item = item[:n_idx]+'...'+item[n_idx+1:]
        encode_dict[key]=item

# 왜 이 짓을 하는가? 파파고는 한 문장당 200자만 번역이 가능하다 ..
        

In [72]:
encode_dict[0]

'〈Something〉는 VJV이 쓰고 VKV가 1969년 앨범 《Abbey Road》에 담은 노래다.'

In [141]:
import itertools
for idx,value in dict(itertools.islice(encode_dict.items(), 0,10)).items():
    print(idx,value)


0 〈Something〉는 VJV이 쓰고 VKV가 1969년 앨범 《Abbey Road》에 담은 노래다.
1 호남이 기반인 바른미래당·VJV·VKV이 우여곡절 끝에 합당해 민생당(가칭)으로 재탄생한다.
2 K리그2에서 성적 1위를 달리고 있는 VKV는 지난 26일 VJV으로부터 관중 유치 성과와 마케팅 성과를 인정받아 ‘풀 스타디움상’과 ‘플러스 스타디움상’을 수상했다.
3 균일가 생활용품점 (주)VKV(대표 VJV)는 코로나19 바이러스로 어려움을 겪고 있는 대구광역시에 행복박스를 전달했다고 10일 밝혔다.
4 VJV년 프로 야구 드래프트 1순위로 VKV에게 입단하면서 등번호는 8번으로 배정되었다.
5 : 유엔, 유럽 의회, VKV (VJV), 국제이주기구, 세계 보건 기구 (WHO), 지중해 연합, 이슬람 협력 기구, 유럽 안보 협력 기구, 국제 통화 기금, 세계무역기구 그리고 프랑코포니.
6 그에 따라 나폴리와 계약을 연장한 마라도나는 VJV년 팀을 UEFA컵 정상으로 인도했으며 이듬해에는 유럽 챔피언 VKV을 상대로 승리를 거두고 다시 한 번 세리에A에서 정상에 등극했다.
7 VKV(朴容旿, VJV(음력 3월 19일)(음력 3월 19일) ~ 2009년 11월 4일)는 서울에서 태어난 대한민국의 기업인으로 두산그룹 회장, KBO 총재 등을 역임했다.
8 중공군에게 온전히 대항할 수 없을 정도로 약해진 국민당은 VJV로 수도를 옮기는 것을 결정해, 남아있는 VKV군의 병력이나 국가, 개인의 재산등을 속속 타이완으로 옮기기 시작해, 12월에는 중앙 정부 기구도 모두 이전해 타이베이 시를 중화민국의 새로운 수도로 삼았다.
9 특히 김동연 전 경제부총리를 비롯한 김두관 국회의원, VKV 국회의원, 김종민 국회의원, 오제세 국회의원, 최운열 국회의원, 김정우 국회의원, 권칠승 국회의원, 맹성규 국회의원등 VJV 국회의원 8명이 영상 축하 메세지를 보내 눈길을 끌었다.


In [142]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
import numpy as np
from tqdm.auto import tqdm
from datetime import datetime
import pickle
import itertools
# 크롬창(웹드라이버) 열기
options = Options()
options.add_argument("--no-sandbox") #bypass OS security model
options.add_argument("--headless")
options.add_argument("--disable-dev-shm-usage") #overcome limited resource problems
s=Service('/opt/ml/chromedriver')
driver = webdriver.Chrome(options=options, service=s)

    



In [143]:
target_present = EC.presence_of_element_located((By.XPATH, '//*[@id="txtTarget"]'))

# 파파고 페이지 접속
start_index = 0
last_index = 10
kor_to_eng_dict={}
for key, value in dict(itertools.islice(encode_dict.items(), start_index,last_index)).items():
    # 입력창에 검색어 입력
    backtrans=''
    # element=WebDriverWait(driver, 10).until(target_present)
    driver.get('https://papago.naver.com/?sk=ko&tk=en')
    time.sleep(0.1)
    driver.find_element(By.XPATH,value='//*[@id="sourceEditArea"]/label').send_keys(value)
    driver.find_element(By.XPATH,value='//*[@id="btnTranslate"]').click()
    time.sleep(1.5)
    backtrans = WebDriverWait(driver, 20).until(target_present).text
    # backtrans = driver.find_element(By.XPATH,value='//*[@id="txtTarget"]').text
    kor_to_eng_dict[key]=backtrans

with open(f'kor_eng_{start_index}_to_{last_index}.pickle', 'wb') as f:
    pickle.dump(kor_to_eng_dict, f, pickle.HIGHEST_PROTOCOL)

In [144]:
kor_to_eng_dict # 아무리 살리려고 해도 지워지는 경우가 있다.

{0: "Something is written by VJV and included in VKV's 1969 album Abbey Road.",
 1: "The Bareunmirae Party, VJV, and VKV, which are based in Honam, will merge after many twists and turns and be reborn as the People's Livelihood Party (tentative name).",
 2: 'VKV, which ranks first in the K League 2, won the "Full Stadium Award" and the "Plus Stadium Award" in recognition of its audience attraction and marketing performance by VJV on the 26th.',
 3: 'VKV Co., Ltd. (CEO VJV) announced on the 10th that it has delivered a happiness box to Daegu Metropolitan City, which is suffering from the COVID-19 virus.',
 4: 'When he joined VKV as the No. 1 professional baseball draft in VJV, his uniform number was assigned No. 8.',
 5: ': The United Nations, the European Parliament, the VKV (VJV), the International Organization for Migration, the World Health Organization (WHO), the Mediterranean Union, the Islamic Cooperation Organization, the European Security Cooperation Organization, the Internati

In [77]:
target_present = EC.presence_of_element_located((By.XPATH, '//*[@id="txtTarget"]'))
eng_to_kor_dict={}

for key,data in kor_to_eng_dict.items():
    # 입력창에 검색어 입력
    backtrans=''
    # element=WebDriverWait(driver, 10).until(target_present)
    driver.get('https://papago.naver.com/?sk=en&tk=ko')
    time.sleep(0.1)
    driver.find_element(By.XPATH,value='//*[@id="sourceEditArea"]/label').send_keys(data)
    driver.find_element(By.XPATH,value='//*[@id="btnTranslate"]').click()
    time.sleep(1.5)
    backtrans = WebDriverWait(driver, 20).until(target_present).text
    # backtrans = driver.find_element(By.XPATH,value='//*[@id="txtTarget"]').text
    eng_to_kor_dict[key]=backtrans

with open(f'eng_kor_{start_index}_to_{last_index}.pickle', 'wb') as f:
    pickle.dump(eng_to_kor_dict, f, pickle.HIGHEST_PROTOCOL)
driver.close()

In [139]:
eng_to_kor_dict

{0: '어떤 것은 VJV에 의해 쓰여졌고 VKV의 1969년 앨범 Abbey Road에 포함되어 있다.',
 1: '호남을 기반으로 하는 바른정당과 VJV, VKV가 우여곡절 끝에 합당해 민생당(가칭)으로 거듭난다.',
 2: '',
 3: '주식회사 VKV(대표 VJV)는 COVID-19 바이러스로 몸살을 앓고 있는 대구광역시에 행복상자를 전달했다고 10일 밝혔다.',
 4: '그가 VJV 프로야구 드래프트 1순위로 VKV에 입단했을 때, 그의 유니폼 번호는 8번이었다.',
 5: ': 유엔, 유럽의회, VKV(VJV), 국제이주기구, 세계보건기구(WHO), 지중해연합, 이슬람협력기구, 유럽안보협력기구, 국제통화기금, 세계무역기구.',
 6: '이에 따라 나폴리와 계약을 연장한 마라도나는 VJV팀을 UEFA컵 정상에 올려놓았고 이듬해 유럽 챔피언 VKV를 꺾고 다시 한 번 세리에A 정상에 올랐다.',
 7: 'VKV (음력 3월 19일) - 2009년 11월 4일 서울에서 태어난 대한민국 기업인으로 두산그룹 회장 및 KBO 총재를 역임하였다.',
 8: '중국군에 대해 너무 약해진 국민당은 수도를 VJV로 옮기기로 결정하고 12월부터 남아 있는 VKV 병력과 국유재산을 차례로 대만으로 이전하기 시작했고, 12월에는 중앙정부가 타이베이시를 옮겨 중화민국의 새 수도가 되었다.',
 9: '특히 김동연 전 부총리, VKV 의원, 김종민 오세 최운열 김정우 권칠승 맹성규 의원 등 8명의 VJV 의원이 영상 메시지를 보냈다.',
 10: 'VKV는 VJV 클럽 태어스의 청소년 팀을 위해 축구를 시작했다.',
 11: '1990년대 중반 이후 태어난 세대의 대표적 아이콘으로 알려진 VJV VKV(본명 빌리 오코넬19)가 팝의 새로운 역사를 썼다.',
 12: '2009년 9월 미국 프로야구 VKV 소속 야구선수 VJV는 MBC 스페셜에서 박찬호가 당신을 잊지 않고 최진실의 아픔과 죽음을 이해한다. 나는 최진실의 사건에 눈물을 흘렸다. 나는 한번 죽기로 결심한 적이 있

In [145]:
aug_dict ={}
for i in range(1):
    with open(f'eng_kor_{i*(last_index-start_index)}_to_{(i+1)*(last_index-start_index)}.pickle','rb') as f:
        data = pickle.load(f)
        aug_dict.update(data)
aug_dict

{0: '어떤 것은 VJV에 의해 쓰여졌고 VKV의 1969년 앨범 Abbey Road에 포함되어 있다.',
 1: '호남을 기반으로 하는 바른정당과 VJV, VKV가 우여곡절 끝에 합당해 민생당(가칭)으로 거듭난다.',
 2: '',
 3: '주식회사 VKV(대표 VJV)는 COVID-19 바이러스로 몸살을 앓고 있는 대구광역시에 행복상자를 전달했다고 10일 밝혔다.',
 4: '그가 VJV 프로야구 드래프트 1순위로 VKV에 입단했을 때, 그의 유니폼 번호는 8번이었다.',
 5: ': 유엔, 유럽의회, VKV(VJV), 국제이주기구, 세계보건기구(WHO), 지중해연합, 이슬람협력기구, 유럽안보협력기구, 국제통화기금, 세계무역기구.',
 6: '이에 따라 나폴리와 계약을 연장한 마라도나는 VJV팀을 UEFA컵 정상에 올려놓았고 이듬해 유럽 챔피언 VKV를 꺾고 다시 한 번 세리에A 정상에 올랐다.',
 7: 'VKV (음력 3월 19일) - 2009년 11월 4일 서울에서 태어난 대한민국 기업인으로 두산그룹 회장 및 KBO 총재를 역임하였다.',
 8: '중국군에 대해 너무 약해진 국민당은 수도를 VJV로 옮기기로 결정하고 12월부터 남아 있는 VKV 병력과 국유재산을 차례로 대만으로 이전하기 시작했고, 12월에는 중앙정부가 타이베이시를 옮겨 중화민국의 새 수도가 되었다.',
 9: '특히 김동연 전 부총리, VKV 의원, 김종민 오세 최운열 김정우 권칠승 맹성규 의원 등 8명의 VJV 의원이 영상 메시지를 보냈다.',
 10: 'VKV는 VJV 클럽 태어스의 청소년 팀을 위해 축구를 시작했다.',
 11: '1990년대 중반 이후 태어난 세대의 대표적 아이콘으로 알려진 VJV VKV(본명 빌리 오코넬19)가 팝의 새로운 역사를 썼다.',
 12: '2009년 9월 미국 프로야구 VKV 소속 야구선수 VJV는 MBC 스페셜에서 박찬호가 당신을 잊지 않고 최진실의 아픔과 죽음을 이해한다. 나는 최진실의 사건에 눈물을 흘렸다. 나는 한번 죽기로 결심한 적이 있

In [151]:
aug_dict

{0: '어떤 것은 조지 해리슨에 의해 쓰여졌고 비틀즈의 1969년 앨범 Abbey Road에 포함되어 있다.',
 1: '호남을 기반으로 하는 바른정당과 대안신당, 민주평화당가 우여곡절 끝에 합당해 민생당(가칭)으로 거듭난다.',
 2: '',
 3: '주식회사 아성다이소(대표 박정부)는 COVID-19 바이러스로 몸살을 앓고 있는 대구광역시에 행복상자를 전달했다고 10일 밝혔다.',
 4: '그가 1967 프로야구 드래프트 1순위로 요미우리 자이언츠에 입단했을 때, 그의 유니폼 번호는 8번이었다.',
 5: ': 유엔, 유럽의회, 북대서양 조약 기구(NATO), 국제이주기구, 세계보건기구(WHO), 지중해연합, 이슬람협력기구, 유럽안보협력기구, 국제통화기금, 세계무역기구.',
 6: '이에 따라 나폴리와 계약을 연장한 마라도나는 1989팀을 UEFA컵 정상에 올려놓았고 이듬해 유럽 챔피언 AC 밀란를 꺾고 다시 한 번 세리에A 정상에 올랐다.',
 7: 'VKV (음력 3월 19일) - 2009년 11월 4일 서울에서 태어난 대한민국 기업인으로 두산그룹 회장 및 KBO 총재를 역임하였다.',
 8: '중국군에 대해 너무 약해진 국민당은 수도를 타이베이로 옮기기로 결정하고 12월부터 남아 있는 중화민국 병력과 국유재산을 차례로 대만으로 이전하기 시작했고, 12월에는 중앙정부가 타이베이시를 옮겨 중화민국의 새 수도가 되었다.',
 9: '특히 김동연 전 부총리, 안규백 의원, 김종민 오세 최운열 김정우 권칠승 맹성규 의원 등 8명의 더불어민주당 의원이 영상 메시지를 보냈다.',
 10: '하비에르 파스토레는 아르헨티나 클럽 태어스의 청소년 팀을 위해 축구를 시작했다.',
 11: '1990년대 중반 이후 태어난 세대의 대표적 아이콘으로 알려진 싱어송라이터 빌리 아일리시(본명 빌리 오코넬19)가 팝의 새로운 역사를 썼다.',
 12: '2009년 9월 미국 프로야구 필라델피아 필리스 소속 야구선수 박찬호는 MBC 스페셜에서 박찬호가 당신을 잊지 않고 최진실의 아

In [147]:
# 인코딩 과정에서 소문자가 나올 수 있어서 고려함
score = 0
aug_subj_idx_dict = {}
aug_obj_idx_dict = {}

for idx,item in aug_dict.items():
    if item != '': # 번역이 안된 경우 제외
        if sub_code in item or sub_code.lower() in item:
            if obj_code in item or obj_code.lower() in item:
                score += 1
                
                sub_start_idx = item.find(sub_code)
                if sub_start_idx == -1:
                    sub_start_idx = item.find(sub_code.lower())
                obj_start_idx = item.find(obj_code)
                if obj_start_idx == -1:
                    obj_start_idx = item.find(obj_code.lower())

                if sub_start_idx < obj_start_idx:
                    item=item.replace(sub_code,sen_dict['subj_word'][idx])
                    item=item.replace(sub_code.lower(),sen_dict['subj_word'][idx])
                    aug_subj_idx_dict[idx]=((sub_start_idx,sub_start_idx+len(sen_dict['subj_word'][idx])-1))

                    obj_start_idx = item.find(obj_code)
                    if obj_start_idx == -1:
                        obj_start_idx = item.find(obj_code.lower())

                    item=item.replace(obj_code,sen_dict['obj_word'][idx])
                    item=item.replace(obj_code.lower(),sen_dict['obj_word'][idx])
                    aug_obj_idx_dict[idx]=((obj_start_idx,obj_start_idx+len(sen_dict['obj_word'][idx])-1))

                else :
                    item=item.replace(obj_code,sen_dict['obj_word'][idx])
                    item=item.replace(obj_code.lower(),sen_dict['obj_word'][idx])
                    aug_obj_idx_dict[idx]=((obj_start_idx,obj_start_idx+len(sen_dict['obj_word'][idx])-1))

                    sub_start_idx = item.find(sub_code)
                    if sub_start_idx == -1:
                        sub_start_idx = item.find(sub_code.lower())

                    item=item.replace(sub_code,sen_dict['subj_word'][idx])
                    item=item.replace(sub_code.lower(),sen_dict['subj_word'][idx])
                    aug_subj_idx_dict[idx]=((sub_start_idx,sub_start_idx+len(sen_dict['subj_word'][idx])-1))

                aug_dict[idx]=item
print("restoration percent : ", score/len(aug_dict))
aug_subj_idx_dict

restoration percent :  0.85


{0: (22, 24),
 1: (24, 28),
 3: (5, 9),
 4: (23, 31),
 5: (12, 21),
 6: (59, 63),
 8: (52, 55),
 9: (14, 16),
 10: (0, 8),
 11: (42, 48),
 12: (17, 25),
 13: (0, 3),
 14: (47, 51),
 16: (0, 2),
 17: (0, 3),
 18: (0, 2),
 19: (71, 77)}

In [153]:
for idx,item in aug_dict.items():
    if idx in aug_subj_idx_dict.keys():
        print(aug_dict[idx][aug_subj_idx_dict[idx][0]:aug_subj_idx_dict[idx][1]+1])

비틀즈
민주평화당
아성다이소
요미우리 자이언츠
북대서양 조약 기구
AC 밀란
중화민국
안규백
하비에르 파스토레
빌리 아일리시
필라델피아 필리스
뉴질랜드
미켈란젤로
이정미
제2총군
문성민
한국수자원공사


In [160]:
aug_data={}
for idx,item in aug_dict.items():
    if idx in aug_subj_idx_dict.keys():
        insert_data={'id':idx+32470,'subj_type':sen_dict['subj_type'][idx],'subj_word':sen_dict['subj_word'][idx],'subj_start':aug_subj_idx_dict[idx][0],'subj_end':aug_subj_idx_dict[idx][1],'obj_type':sen_dict['obj_type'][idx],'obj_word':sen_dict['obj_word'][idx],'obj_start':aug_obj_idx_dict[idx][0],'obj_end':aug_obj_idx_dict[idx][1],'sentence':aug_dict[idx],'label':sen_dict['label'][idx],'label_num':sen_dict['label_num'][idx]}
        train_df_2 = train_df_2.append(insert_data,ignore_index=True)

In [162]:
train_df_2.tail(20) #추가됬는지 확인
# 약간 문제가 되는게 번역이 안되는 문장에 대해 index와 id가 안맞는 일이 발생함 조금 수정하면 될 것 같은데 문제 없는 이상 이대로 하겠음.

,id,subj_type,subj_word,subj_start,subj_end,obj_type,obj_word,obj_start,obj_end,sentence,label,label_num
32467,32467,ORG,완도군,0,2,PER,신우철,7,9,완도군(군수 신우철)이 국토교통부에서 실시한 '2019 교통문화지수 실태조사'에서 229개 기초지자체 중 최상위 등급인 A등급을 받으면서 전국에서 가장 높은 교통안전 인식 수준을 갖고 있는 것으로 나타났다.,org:top_members/employees,1
32468,32468,ORG,JTBC,6,9,ORG,중앙홀딩스,21,25,"중앙일보, JTBC 회장을 지낸 이후 중앙홀딩스 회장, 재단법인 한반도평화만들기 이사장, 한국기원 총재를 맡고 있다.",no_relation,0
32469,32469,ORG,화순군,0,2,PER,구충곤,7,9,"화순군(군수 구충곤)은 17일 동면의 이장 20여 명이 코로나 19 예방을 위해 버스 승강장, 공중화장실 등 다중이용시설과 오동리 천운아파트 주변을 소독하는 방역 봉사활동을 펼쳤다고 밝혔다.",org:top_members/employees,1
32470,32470,ORG,비틀즈,22,24,PER,조지 해리슨,6,11,어떤 것은 조지 해리슨에 의해 쓰여졌고 비틀즈의 1969년 앨범 Abbey Road에 포함되어 있다.,no_relation,0
32471,32471,ORG,민주평화당,24,28,ORG,대안신당,18,21,"호남을 기반으로 하는 바른정당과 대안신당, 민주평화당가 우여곡절 끝에 합당해 민생당(가칭)으로 거듭난다.",no_relation,0
32472,32473,ORG,아성다이소,5,9,PER,박정부,14,16,주식회사 아성다이소(대표 박정부)는 COVID-19 바이러스로 몸살을 앓고 있는 대구광역시에 행복상자를 전달했다고 10일 밝혔다.,org:top_members/employees,1
32473,32474,ORG,요미우리 자이언츠,23,31,DAT,1967,3,6,"그가 1967 프로야구 드래프트 1순위로 요미우리 자이언츠에 입단했을 때, 그의 유니폼 번호는 8번이었다.",no_relation,0
32474,32475,ORG,북대서양 조약 기구,12,21,ORG,NATO,23,26,": 유엔, 유럽의회, 북대서양 조약 기구(NATO), 국제이주기구, 세계보건기구(WHO), 지중해연합, 이슬람협력기구, 유럽안보협력기구, 국제통화기금, 세계무역기구.",org:alternate_names,5
32475,32476,ORG,AC 밀란,59,63,DAT,1989,25,28,이에 따라 나폴리와 계약을 연장한 마라도나는 1989팀을 UEFA컵 정상에 올려놓았고 이듬해 유럽 챔피언 AC 밀란를 꺾고 다시 한 번 세리에A 정상에 올랐다.,no_relation,0
32476,32478,ORG,중화민국,52,55,LOC,타이베이,24,27,"중국군에 대해 너무 약해진 국민당은 수도를 타이베이로 옮기기로 결정하고 12월부터 남아 있는 중화민국 병력과 국유재산을 차례로 대만으로 이전하기 시작했고, 12월에는 중앙정부가 타이베이시를 옮겨 중화민국의 새 수도가 되었다.",org:place_of_headquarters,7
